In [1]:
# 1. Import libraries:
%matplotlib inline
import os
import quandl
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.optimize as spo
from scipy.stats import kurtosis, skew
import seaborn as sns
# from financial_data import *
import tensorflow as tf
mlp.style.use('seaborn')
quandl.save_key('HtwBLPt3k37yZHTvy15K')

/var/folders/dk/mxqr91nj7135zkb7ptp3mq5c0000gn/T/ipykernel_6766/4242266090.py:16: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  mlp.style.use('seaborn')


In [2]:
sp_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500 = pd.read_html(sp_url, header=0)[0]
sp500.head()



,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [3]:
# Correct invalid dates:
sp500.loc[sp500[sp500['Date added']=='1983-11-30 (1957-03-04)'].index,'Date added'] = '1983-11-30'
sp500.loc[sp500[sp500['Date added']=='2001?'].index,'Date added'] = '2001-01-01'
# Filter firms that entered the index after December 2015:
sp500['Date added'] = pd.to_datetime(sp500['Date added'],format='%Y-%m-%d')
sp500 = sp500[sp500['Date added']<'2007-01-01']
print("The number of stocks in the universe is:", sp500.shape[0])

The number of stocks in the universe is: 241


In [4]:
n_stocks = 2
np.random.seed(1792)
universe_tickers = sp500['Symbol'].unique()
tickers = list(np.random.choice(universe_tickers,replace=False,size=n_stocks))
print(tickers)
# sp500[sp500['Symbol'].isin(portfolio_tickers)]

['DIS', 'BIIB']


In [5]:
# tickers = ['AAPL', 'GOOGL', 'MSFT','SPY']
# start_date='2007-01-01'
# end_date='2022-12-31'
# columns = ['Adj Close', 'Volume']
# # Download data
# data = yf.download(tickers, start=start_date, end=end_date)[columns]

# # 'data' will be a Pandas DataFrame containing the historical data for the specified tickers
# print(data)


def download_data(tickers, start_date, end_date, columns):
    data = yf.download(tickers, start=start_date, end=end_date)
    data=data[columns]
    return data

# Define the tickers, start date, end date, and columns you want to download
# tickers = ['AAPL', 'GOOGL', 'MSFT']
start_date = '2007-01-01'
end_date = '2022-12-31'
columns = [ 'Adj Close', 'Volume','Open','High','Low']

# Download the data
data = download_data(tickers, start_date, end_date, columns)
print(data)


def one_lvl_colnames(df,cols,tickers):
    """This function changes a multi-level column indexation into a one level
    column indexation

    Inputs:
    -------
    df (pandas Dataframe): dataframe with the columns whose indexation will be 
        flattened.
    tickers (list|string): list/string with the tickers (s) in the data frame df.
    cols (list|string): list/string with the name of the columns (e.g. 'Adj Close',
        'High', 'Close', etc.) that are in the dataframe df.
    
    Ouputs:
    -------
    df (pandas Dataframe): dataframe with the same information as df, but 
        with one level of indexation.
    """
    
    
    print("the dataframe before the columns were renamed")
#     print(df.columns)
    df_not_renamed=df.copy()
#     df_not_renamed.drop("Open",inplace=True)
    print(df_not_renamed.columns)
    # Define important variables:
    if isinstance(tickers, str):
        tickers = [tickers]
    if isinstance(cols, str):
        
        cols = [cols]
        print(cols)

    # For multi-level column indexing:
    if isinstance(df.columns.values[0], tuple):

        # Define important varibles
        columns = df.columns.values
        new_cols = []

        # Itarate through the multi-level column names and flatten them:
        for col in columns:
            temp = []
            for name in col:
                if name != '':
                    temp.append(name)
            new_temp = '_'.join(temp)
            new_cols.append(new_temp)
        
        # Change the column names:
        df.columns = new_cols
        print(df.columns)
    
    # For uni-level colum indexing:
    elif isinstance(df.columns.values[0], str):
        
        # Define new names:
        col_names = [column+'_'+ticker for column in cols\
                     for ticker in tickers]
        df.columns = col_names
    
    print("the dataframe after the columns have been renamed") 
#     print(df)

    return df

df2=pd.DataFrame(one_lvl_colnames(data,cols=columns,tickers=tickers))
df2_columns=list(df2.columns)
df2.columns = [col.replace('Adj Close', 'Close') for col in df2_columns]
df2.fillna(method='ffill',inplace=True)
df2.fillna(method='bfill',inplace=True)
df2

[*********************100%%**********************]  2 of 2 completed
             Adj Close              Volume                  Open             \
                  BIIB        DIS     BIIB       DIS        BIIB        DIS   
Date                                                                          
2007-01-03   49.330002  28.317097  3833500  13562595   49.279999  33.748165   
2007-01-04   49.750000  28.540651  2884300   9806285   49.270000  33.738300   
2007-01-05   49.759998  28.308817  2230400  10551445   49.990002  33.807354   
2007-01-08   50.020000  28.565491  3160200   9479676   50.000000  33.728436   
2007-01-09   49.500000  28.524090  2816500  11588444   49.880001  34.034248   
...                ...        ...      ...       ...         ...        ...   
2022-12-23  279.160004  88.010002   624800  11171600  280.450012  86.059998   
2022-12-27  274.769989  86.370003   638600  11561400  279.890015  87.419998   
2022-12-28  274.040009  84.169998   521000  12399500  275.6400

,Close_BIIB,Close_DIS,Volume_BIIB,Volume_DIS,Open_BIIB,Open_DIS,High_BIIB,High_DIS,Low_BIIB,Low_DIS
Date,,,,,,,,,,
2007-01-03,49.330002,28.317097,3833500,13562595,49.279999,33.748165,50.250000,34.073711,48.200001,33.531136
2007-01-04,49.750000,28.540651,2884300,9806285,49.270000,33.738300,50.099998,34.083576,48.880001,33.708706
2007-01-05,49.759998,28.308817,2230400,10551445,49.990002,33.807354,50.330002,33.975060,49.509998,33.531136
2007-01-08,50.020000,28.565491,3160200,9479676,50.000000,33.728436,50.160000,34.162495,48.910000,33.610054
2007-01-09,49.500000,28.524090,2816500,11588444,49.880001,34.034248,50.139999,34.221684,49.299999,33.491676
...,...,...,...,...,...,...,...,...,...,...
2022-12-23,279.160004,88.010002,624800,11171600,280.450012,86.059998,280.450012,88.070000,276.070007,85.769997
2022-12-27,274.769989,86.370003,638600,11561400,279.890015,87.419998,279.890015,87.940002,273.380005,85.959999
2022-12-28,274.040009,84.169998,521000,12399500,275.640015,86.080002,276.920013,86.690002,272.640015,84.070000


In [6]:
def momentum(prices):
    """This function finds the momentum metric for a group of assets.
    Inputs:
    -------
    prices (pandas dataframe|series): dataframe with the information of the
        prices of a group of assets or an indivitual asset.
    
    Outputs:
    --------
    momentum_df (pandas Dataframe): dataframe with the information of the
        momentum.
    """
    # Compute components:
    first = prices.iloc[0]
    last = prices.iloc[-1]

    # Compute momentum:
    momentum_df = last/first

    return momentum_df

def simple_moving_average(prices):
    """This function computes the simple moving average for a set of prices
    given a time window.

    Inputs:
    -------
    prices (pandas Dataframe|Series): dataframe with the information of the
        prices of the analyzed assets.

    Outputs:
    --------
    sma (pandas Dataframe): dataframe of the SMA of the assets' prices in
        the prices dataframe.
    """
    # Compute the SMA of the assets:
    mean = prices.mean()
    sma = prices[-1]/mean-1

    return sma
    
def bollinger_bands(prices):
    """This function computes the bollinger bands values for a set of assets
    returned in a pandas dataframe.

    Inputs:
    -------
    prices (pandas Dataframe|Series): dataframe with the price infromation
        of the assets analyzed.
    window (numeric value, default=None): determines if compute the BBs in
        a rolling manner, or if compute them to the whole input of prices.
    
    Outputs:
    --------
    bb (pandas dataframe): dataframe with the information of the bollinger
        bands of the assets in the prices dataframe.
    """
    # Compute components:
    ma = prices.mean()
    std = prices.std()

    # Compute bollinger bands:
    bb = (prices[-1]-ma)/(2*std)

    return bb


In [7]:
def rolling_statistics(df,cols='Adj Close',tickers=None,functions=None,
                      window=20,bollinger=False,roll_linewidth=1.5,**kwargs):
        '''This method extracts the rolling statistics from a time series, and
        can plot the rolling window with the data, adding the Bollinger bands.
        
        Inputs:
        -------
        column (string, default=None): the column from which you want compute 
            the rolling function.
        tickers (str|list, default=None): the ticker(s) from which you want to know the 
            information.
        functions (function|string|list): function(s) that will be rolled through the 
            time series.
        window (int): the window of the rolling data
            
        OUTPUTS:
            rolled (pandas series): a series with the rolling statistics specified

        '''
        # Define important varibles:
#         df = self.df
#         if isinstance(tickers,type(None)):
#             tickers = self.get_tickers()
        col_names = cols
        if isinstance(functions,type(None)):
            functions = [momentum, simple_moving_average, bollinger_bands]
        elif not isinstance(functions,list):
            functions = [functions]

        # Define the actual dataframe analized
        df = df[col_names]
#         print(df)

        # Compute the rolling statistics:
        rolling_stats = df.rolling(window).agg(functions)
        
        return rolling_stats
        
#         print(rolling_stats)


In [8]:
import pandas as pd

# Assuming df is your dataframe
# If it's not loaded yet, you should load it using pd.read_csv() or any other method

# Get a list of unique tickers
tickers = df2.columns.str.split('_').str[1].unique()

# Create a dictionary to store information for each ticker
ticker_data = {}

# Loop through each ticker and extract the information
for ticker in tickers:
    ticker_info = {}
    
    # Iterate through the columns and filter based on the current ticker
    for column in df2.columns:
        if ticker in column:
            ticker_info[column] = df2[column]
    
    # Store the ticker information in the dictionary
    ticker_data[ticker] = ticker_info

# Now, you can access information for each ticker using ticker_data dictionary
# For example, to get the Adj Close data for BIIB, you can use:
biib = pd.DataFrame(ticker_data['BIIB'])
biib.columns=biib.columns.str.split('_').str[0].unique()

In [9]:
biib
from stockstats import StockDataFrame

stock_df_biib = StockDataFrame.retype(biib)
stock_df_biib

stock_df_biib.init_all()

In [10]:
stock_df_biib

,close,volume,open,high,low,rate,middle,tp,boll,boll_ub,...,pgo,psl,rsi,rsv,stochrsi,tema,trix,wr,vr,vwma
Date,,,,,,,,,,,,,,,,,,,,,
2007-01-03,49.330002,3833500,49.279999,50.250000,48.200001,NaN,49.260001,49.260001,49.330002,NaN,...,0.000000,0.000000,NaN,55.122024,NaN,49.330002,0.000000,-44.877976,100.000000,49.260001
2007-01-04,49.750000,2884300,49.270000,50.099998,48.880001,0.851405,49.576667,49.576667,49.540001,50.133968,...,0.130812,8.333333,100.000000,75.609747,NaN,49.723120,0.135311,-24.390253,250.478675,49.395962
2007-01-05,49.759998,2230400,49.990002,50.330002,49.509998,0.020097,49.866666,49.866666,49.613333,50.104180,...,0.112357,16.666667,100.000000,73.239285,NaN,49.783191,0.101156,-26.760715,366.842311,49.513288
2007-01-08,50.020000,3160200,50.000000,50.160000,48.910000,0.522512,49.696667,49.696667,49.715000,50.285963,...,0.236725,25.000000,100.000000,85.445952,NaN,49.993999,0.096743,-14.554048,531.715143,49.561149
2007-01-09,49.500000,2816500,49.880001,50.139999,49.299999,-1.039585,49.646666,49.646666,49.672000,50.202546,...,-0.146830,25.000000,52.997540,61.032797,0.000000,49.650845,0.070253,-38.967203,215.320340,49.577287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,279.160004,624800,280.450012,280.450012,276.070007,-0.516725,278.560008,278.560008,289.894499,303.001943,...,-1.222242,33.333333,40.154053,16.595044,0.000000,279.813517,-0.120751,-83.736861,74.243877,286.591810
2022-12-27,274.769989,638600,279.890015,279.890015,273.380005,-1.572580,276.013336,276.013336,289.037999,303.735876,...,-1.722403,33.333333,36.396562,6.682620,0.000000,275.683329,-0.148469,-93.477314,67.014197,285.819225
2022-12-28,274.040009,521000,275.640015,276.920013,272.640015,-0.265670,274.533346,274.533346,288.169499,304.263686,...,-1.706732,33.333333,35.796714,8.615347,0.000000,273.629693,-0.177789,-93.650818,61.135674,285.046076


In [11]:
stock_df_biib.shape

(4028, 74)

In [12]:
pd.set_option('display.max_rows', 100)

In [23]:
import pandas as pd

# Assuming df is your dataframe with features and target variable
# If it's not loaded yet, you should load it using pd.read_csv() or any other method

# Calculate the correlation matrix
stock_df_biib_corr=stock_df_biib.copy()
stock_df_biib_corr.dropna(inplace=True)
correlation_matrix = stock_df_biib_corr.corr()

# Stack the correlation matrix to create a DataFrame
correlation_df = correlation_matrix.stack().reset_index()
correlation_df.columns = ['Feature1', 'Feature2', 'Correlation']
correlation_df = correlation_df[(correlation_df['Feature1'] != correlation_df['Feature2']) 
                                & (correlation_df['Feature1'] < correlation_df['Feature2'])]

correlation_df=pd.DataFrame(correlation_df.sort_values(by='Correlation', ascending=False).reset_index(drop=True))
# Print the resulting DataFrame
print("DataFrame from Pairwise Correlation:")
# print(correlation_df.loc[150:200])
print(correlation_df.head(100))



# Print the sorted correlations
# print(pd.DataFrame(correlation_matrix))

DataFrame from Pairwise Correlation:
         Feature1       Feature2  Correlation
0          change           rate     1.000000
1          middle             tp     1.000000
2           close         middle     0.999902
3           close             tp     0.999902
4            tema             tp     0.999860
5          middle           tema     0.999860
6           close           tema     0.999803
7            high             tp     0.999777
8            high         middle     0.999777
9             low             tp     0.999718
10            low         middle     0.999718
11            low           open     0.999653
12          close           high     0.999628
13            low           tema     0.999611
14           high           tema     0.999565
15         middle           open     0.999532
16           open             tp     0.999532
17          close            low     0.999476
18           open           tema     0.999442
19           high           open     0.9992

In [34]:
stock_df_biib
df_copy=stock_df_biib.copy()

# features_to_shift = [col for col in df_copy.columns if col != 'close']
features_to_shift=['close']
# feature_list=df_copy.columns
df_copy[features_to_shift] = df_copy[features_to_shift].apply(lambda x: x.shift(1))
# df_copy=df_copy.iloc[1:]
df_copy

,close,volume,open,high,low,rate,middle,tp,boll,boll_ub,...,pgo,psl,rsi,rsv,stochrsi,tema,trix,wr,vr,vwma
Date,,,,,,,,,,,,,,,,,,,,,
2007-01-03,NaN,3833500,49.279999,50.250000,48.200001,NaN,49.260001,49.260001,49.330002,NaN,...,0.000000,0.000000,NaN,55.122024,NaN,49.330002,0.000000,-44.877976,100.000000,49.260001
2007-01-04,49.330002,2884300,49.270000,50.099998,48.880001,0.851405,49.576667,49.576667,49.540001,50.133968,...,0.130812,8.333333,100.000000,75.609747,NaN,49.723120,0.135311,-24.390253,250.478675,49.395962
2007-01-05,49.750000,2230400,49.990002,50.330002,49.509998,0.020097,49.866666,49.866666,49.613333,50.104180,...,0.112357,16.666667,100.000000,73.239285,NaN,49.783191,0.101156,-26.760715,366.842311,49.513288
2007-01-08,49.759998,3160200,50.000000,50.160000,48.910000,0.522512,49.696667,49.696667,49.715000,50.285963,...,0.236725,25.000000,100.000000,85.445952,NaN,49.993999,0.096743,-14.554048,531.715143,49.561149
2007-01-09,50.020000,2816500,49.880001,50.139999,49.299999,-1.039585,49.646666,49.646666,49.672000,50.202546,...,-0.146830,25.000000,52.997540,61.032797,0.000000,49.650845,0.070253,-38.967203,215.320340,49.577287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,280.609985,624800,280.450012,280.450012,276.070007,-0.516725,278.560008,278.560008,289.894499,303.001943,...,-1.222242,33.333333,40.154053,16.595044,0.000000,279.813517,-0.120751,-83.736861,74.243877,286.591810
2022-12-27,279.160004,638600,279.890015,279.890015,273.380005,-1.572580,276.013336,276.013336,289.037999,303.735876,...,-1.722403,33.333333,36.396562,6.682620,0.000000,275.683329,-0.148469,-93.477314,67.014197,285.819225
2022-12-28,274.769989,521000,275.640015,276.920013,272.640015,-0.265670,274.533346,274.533346,288.169499,304.263686,...,-1.706732,33.333333,35.796714,8.615347,0.000000,273.629693,-0.177789,-93.650818,61.135674,285.046076


In [32]:
import pandas as pd

# Assuming df is your dataframe with features and target variable
# If it's not loaded yet, you should load it using pd.read_csv() or any other method

# Calculate the correlation matrix
stock_df_biib_corr=df_copy.copy()
stock_df_biib_corr.ffill(inplace=True)
stock_df_biib_corr.bfill(inplace=True)

print(stock_df_biib_corr)
correlation_matrix = stock_df_biib_corr.corr()

# Assuming 'Target_Variable' is the name of your target variable column
target_correlation = correlation_matrix['close'].abs()

# Sort the correlations in descending order to see the most correlated features
sorted_correlations = target_correlation.sort_values(ascending=False)

# Print the sorted correlations
print(pd.DataFrame(sorted_correlations)[1:])


                 close     volume        open        high         low  \
Date                                                                    
2007-01-04   49.750000  3833500.0   49.279999   50.250000   48.200001   
2007-01-05   49.759998  2884300.0   49.270000   50.099998   48.880001   
2007-01-08   50.020000  2230400.0   49.990002   50.330002   49.509998   
2007-01-09   49.500000  3160200.0   50.000000   50.160000   48.910000   
2007-01-10   49.279999  2816500.0   49.880001   50.139999   49.299999   
...                ...        ...         ...         ...         ...   
2022-12-23  279.160004   946500.0  283.609985  283.649994  278.309998   
2022-12-27  274.769989   624800.0  280.450012  280.450012  276.070007   
2022-12-28  274.040009   638600.0  279.890015  279.890015  273.380005   
2022-12-29  276.000000   521000.0  275.640015  276.920013  272.640015   
2022-12-30  276.920013   594000.0  274.829987  279.140015  274.299988   

                rate      middle          tp      

In [13]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 20)

# vif_data=pd.DataFrame(vif_data)
# vif_data.head(70)

In [24]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Assuming df is your dataframe with features
# If it's not loaded yet, you should load it using pd.read_csv() or any other method

# Add a constant term to the features for intercept
vif_df=stock_df_biib.copy()
vif_df.dropna(inplace=True)
vif_df['Intercept'] = 1

# Get a list of feature column names (excluding the target variable)
features = vif_df.drop('close', axis=1).columns.tolist()

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data["Feature"] = features
vif_data["VIF"] = [variance_inflation_factor(vif_df[features].values, i) for i in range(len(features))]

# Print the VIF values
print(vif_data)


/Users/inderkaur/anaconda3/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
/Users/inderkaur/anaconda3/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
/Users/inderkaur/anaconda3/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
/Users/inderkaur/anaconda3/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
/Users/inderkaur/anaconda3/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
/Users/inderkaur/anaconda3/lib

          Feature           VIF
0          volume  2.907731e+00
1            open  4.402955e+03
2            high           inf
3             low           inf
4            rate           inf
5          middle           inf
6              tp           inf
7            boll           inf
8         boll_ub           inf
9         boll_lb           inf
10           macd           inf
11          macds           inf
12          macdh           inf
13            pvo           inf
14           pvos           inf
15           pvoh           inf
16            ppo           inf
17           ppos           inf
18           ppoh           inf
19            qqe           inf
20           qqel           inf
21           qqes           inf
22             cr  9.229197e+00
23         cr-ma1  1.557604e+01
24         cr-ma2  2.273663e+01
25         cr-ma3  8.213803e+00
26             tr  1.499395e+01
27             dx  1.544738e+01
28            adx  7.261776e+01
29           adxr  3.488105e+01
30      